<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML-F1/blob/master/Carnets%20Jupyter/Ressentis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classification de ressentis avec distilBERT**

L'objectif est de créer un modèle qui prend en entrée des commentaires (en Anglais) et attribue à chacun un ressenti positif ou négatif.  
Le modèle est composé de deux parties :  
* DistilBERT va encoder le commentaire et en extraire des informations qui seront passées ensuite au réseau de neurones.  
* Le modèle suivant est un réseau de neurones qui sera créé avec Keras.  

<img src="https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Images/StructureBERT.png" />  
  
  Les données qui s'échangent entre les deux modèles sont des vecteurs de dimension 768. Ces vecteurs sont l'équivalent de l'application d'un algorithme de prolongation lexicale sur les mots qui composent le commentaire.

# **Installation et importations des librairies**


In [2]:
!pip install transformers --quiet

     |████████████████████████████████| 1.1MB 4.7MB/s 
     |████████████████████████████████| 1.1MB 24.5MB/s 
     |████████████████████████████████| 3.0MB 40.6MB/s 
     |████████████████████████████████| 890kB 42.9MB/s 


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, Input, Dropout, Lambda
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

from transformers import TFDistilBertModel
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

import random

# **Importation des données**

On utilise la librairie pandas pour lire les données depuis le fichier csv disponible sur le site de [standford](https://nlp.stanford.edu/sentiment/index.html) qui contient des commentaires sur des films, chacun d'eux avec une note positive (1) ou négative (0).

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/AlexandreBourrieau/ML-F1/master/Carnets%20Jupyter/Donn%C3%A9es/train.csv', delimiter='\t', header=None)

Affiche quelques informations :

In [5]:
print(df[0:10])
print("Total des données : ", str(len(df)))
print("Nombre d'avis positifs et négatifs : ",df[1].value_counts())

                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
3  this is a visually stunning rumination on love...  1
4  jonathan parker 's bartleby should have been t...  1
5  campanella gets the tone just right funny in t...  1
6  a fan film that for the uninitiated plays bett...  0
7  b art and berling are both superb , while hupp...  1
8  a little less extreme than in the past , with ...  0
9                       the film is strictly routine  0
Total des données :  6920
Nombre d'avis positifs et négatifs :  1    3610
0    3310
Name: 1, dtype: int64


# **Préparation des données**

In [6]:
MAX_SEQUENCE_LENGTH = 500

# Chargement des commentaires et des ressentis
commentaires = df[0].astype(str).tolist()                 # Récupère tous les commentaires dans une liste python
ressentis = df[1].tolist()                                # Récupère tous les ressentis dans une liste python
labels = np.asarray(ressentis)                            # Créé un tableau de type numpy avec les ressentis

x_entrainement, x_test, y_entrainement, y_test = train_test_split(commentaires, labels, test_size=0.25)

In [ ]:
# Instanciation du tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Préparation des données d'entrainement
output_tokenizer_entrainement = []
for phrase in x_entrainement:
  output_tokenizer_entrainement.append(tokenizer.encode_plus(phrase,max_length=MAX_SEQUENCE_LENGTH, padding="longest", truncation=True, return_tensors='tf'))

# Préparation des données de tests
output_tokenizer_tests = []
for phrase in x_test:
  output_tokenizer_tests.append(tokenizer.encode_plus(phrase,max_length=MAX_SEQUENCE_LENGTH, padding="longest", truncation=True, return_tensors='tf'))

Regardons un peu comment sont formatées les données :

In [15]:
print("Commentaire original :", x_entrainement[1])
print("Résultat de la tokénisation: ", output_tokenizer_entrainement[1:2])

Commentaire original : one of the best movies of the year
Résultat de la tokénisation:  [{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[ 101, 2028, 1997, 1996, 2190, 5691, 1997, 1996, 2095,  102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}]


Regardons les 5 premiers résutats de la tokénisation : On peut identifier les mot-clés **[CLS]** (valeur : 101) et **[SEP]** (valeur : 102)

In [16]:
output_tokenizer_entrainement[1:5]

[{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
 array([[ 101, 2028, 1997, 1996, 2190, 5691, 1997, 1996, 2095,  102]],
       dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>},
 {'input_ids': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=
 array([[  101,  2664,  2178,  7779,  4316,  2008, 11896,  2000,  2191,
         11706,  2224,  1997,  2010,  3327, 11725,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>},
 {'input_ids': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=
 array([[  101,  1996,  5260,  2057,  2024,  2445,  2182,  2024,  3432,
          2205, 20857,  2000,  2022,  5875,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 15), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>},
 {'input_ids': <tf.Tensor: shape=(1, 26),

# **Définition du modèle**